## DSPY Example

In [12]:
import os
import dspy
from dotenv import load_dotenv
from pprint import pprint

load_dotenv('../../../.env')

True

---

## 1. Simple Setup

In [3]:
lm = dspy.LM(
    'openai/deepseek-chat',      # The 'openai/' prefix indicates an OpenAI-compatible model
    api_key=os.getenv('DEEPSEEK_API_KEY'),
    api_base='https://api.deepseek.com',
    model_type='chat'            # 'chat' because the endpoint expects Chat-style messages.
)

# Let DSPy know we want to use this LM globally
dspy.configure(lm=lm)
print("DSPy is configured to use DeepSeek's 'deepseek-chat'!")

DSPy is configured to use DeepSeek's 'deepseek-chat'!


In [13]:
result = lm("Hello, how are you?")
pprint(result[0])

("Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here "
 'and ready to help you with whatever you need. How are you doing today? 😊')


---

## 2. Loading Data with DSPy DataLoaders

DSPy provides convenient utilities to load data from CSVs, HuggingFace Datasets, or a raw Python list

In [14]:
from dspy.datasets import DataLoader

dl = DataLoader()

# Example 1: Load from HuggingFace
# (Ensure you have 'datasets' library installed via `pip install datasets`)
code_alpaca = dl.from_huggingface("HuggingFaceH4/CodeAlpaca_20K", split=["train", "test"])
print("Splits in CodeAlpaca:", code_alpaca.keys())
print("Number of train examples:", len(code_alpaca['train']))
print("Number of test examples:", len(code_alpaca['test']))

README.md:   0%|          | 0.00/195 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/336k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18019 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2003 [00:00<?, ? examples/s]

Splits in CodeAlpaca: dict_keys(['train', 'test'])
Number of train examples: 18019
Number of test examples: 2003


In [27]:
# Example 2: Load from CSV
# Suppose you have a CSV with columns: instruction, context, response
example_csv = dl.from_csv("../_resources/test.csv", fields=("example_header", "value"))
print("Number of examples in the csv:", len(example_csv ))


Generating train split: 0 examples [00:00, ? examples/s]

Number of examples in the csv: 13


In [28]:
# Example 3: Splitting a Python list of Examples
splits = dl.train_test_split(example_csv, train_size=0.8)
train_dataset = splits['train']
test_dataset = splits['test']
print("train dataset size:", len(train_dataset))
print("test dataset size:", len(test_dataset))

train dataset size: 10
test dataset size: 3


---

## 3. Creating a Simple DSPy Program

DSPy organizes your “AI logic” into modules using OOP (classes and objects)

A module has an input signature and an output signature. 
Here’s a very simple classification example that classifies text into “positive,” “negative,” or “neutral.”

In [29]:
from typing import Literal
import dspy

# 1) Define a signature. This is effectively your structured “prompt+response” contract.
class SentimentSignature(dspy.Signature):
    """
    Classify a given sentence as positive, negative, or neutral.
    """
    sentence: str = dspy.InputField()
    sentiment: Literal['positive','negative','neutral'] = dspy.OutputField()

# 2) Create a DSPy module that implements the signature with a simple approach:
sentiment_classifier = dspy.Predict(SentimentSignature)

# 3) Try it out:
sample_text = "I absolutely love this new phone, but it's a bit expensive."
result = sentiment_classifier(sentence=sample_text)

print(f"Input Sentence: {sample_text}")
print(f"Predicted Sentiment: {result.sentiment}")


Input Sentence: I absolutely love this new phone, but it's a bit expensive.
Predicted Sentiment: positive


---

## Opinion about DSPy


- Known as the "pytorch for llm programs", instead of relying on chains like LangChain, it implements graphs. 
- I don't like how it relies on implicit functionality that isn't clear what is actually happening
- For example: equating the LLM workflow to a neural network code with functions named `forward()` seems like an ML engineers approach to over engineer an LLM powered application.
- I guess some of the utilities are good